In [1]:
import requests
from datetime import datetime as dt
import json
import pandas as pd

In [2]:
# URL base da API histórica do OpenWeatherMap
base_url = 'http://history.openweathermap.org/data/2.5/history/city?'

In [3]:
# API Key
api_key = 'CHAVE-API'

In [4]:
# Latitude, Longitude, Range, Data_inicio, Data_fim, Units
lat= "-22.866070"
long = "-43.403470"
tipo = 'hour'
start = '1669334400'
end = '1696464000'
units = 'metric'

In [5]:
# URL Completa
complete_url = base_url + "lat=" + lat + "&lon=" + long + "&type=" + tipo + "&start=" + start + "&end=" + end + "&units=" + units + "&appid=" + api_key

In [6]:
response = requests.get(complete_url)

In [7]:
x = response.json()
x

{'message': 'Count: 169',
 'cod': '200',
 'city_id': 1,
 'calctime': 0.006584212,
 'cnt': 169,
 'list': [{'dt': 1669334400,
   'main': {'temp': 25.37,
    'feels_like': 26.23,
    'pressure': 1013,
    'humidity': 87,
    'temp_min': 25.37,
    'temp_max': 25.37},
   'wind': {'speed': 0.93, 'deg': 18, 'gust': 1.11},
   'clouds': {'all': 93},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04n'}]},
  {'dt': 1669338000,
   'main': {'temp': 24.25,
    'feels_like': 25.03,
    'pressure': 1013,
    'humidity': 88,
    'temp_min': 24.25,
    'temp_max': 24.25},
   'wind': {'speed': 1.1, 'deg': 328, 'gust': 1.08},
   'clouds': {'all': 67},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04n'}]},
  {'dt': 1669341600,
   'main': {'temp': 20.39,
    'feels_like': 20.81,
    'pressure': 1013,
    'humidity': 89,
    'temp_min': 20.39,
    'temp_max': 20.39},
   'wind': {'speed': 0.83, 'deg

In [8]:
# Extrair a lista de dados climáticos
weather_data = x['list']

In [9]:
# Criação do dataFrame climatico
df = pd.DataFrame(weather_data)
df.head()

,dt,main,wind,clouds,weather,rain
0,1669334400,"{'temp': 25.37, 'feels_like': 26.23, 'pressure...","{'speed': 0.93, 'deg': 18, 'gust': 1.11}",{'all': 93},"[{'id': 804, 'main': 'Clouds', 'description': ...",NaN
1,1669338000,"{'temp': 24.25, 'feels_like': 25.03, 'pressure...","{'speed': 1.1, 'deg': 328, 'gust': 1.08}",{'all': 67},"[{'id': 803, 'main': 'Clouds', 'description': ...",NaN
2,1669341600,"{'temp': 20.39, 'feels_like': 20.81, 'pressure...","{'speed': 0.83, 'deg': 294, 'gust': 0.86}",{'all': 65},"[{'id': 803, 'main': 'Clouds', 'description': ...",NaN
3,1669345200,"{'temp': 24.25, 'feels_like': 25.08, 'pressure...","{'speed': 0.96, 'deg': 301, 'gust': 1.03}",{'all': 77},"[{'id': 803, 'main': 'Clouds', 'description': ...",NaN
4,1669348800,"{'temp': 24.25, 'feels_like': 25.11, 'pressure...","{'speed': 1.2, 'deg': 291, 'gust': 1.29}",{'all': 80},"[{'id': 803, 'main': 'Clouds', 'description': ...",NaN


In [10]:
# Desmembrando a coluna 'main' em colunas separadas
df_main = pd.json_normalize(df['main'])
df = pd.concat([df, df_main], axis=1).drop('main', axis=1)

# Desmembrando a coluna 'wind' em colunas separadas
df_wind = pd.json_normalize(df['wind'])
df = pd.concat([df, df_wind], axis=1).drop('wind', axis=1)

# Desmembrando a coluna 'clouds' em colunas separadas
df_clouds = pd.json_normalize(df['clouds'])
df = pd.concat([df, df_clouds], axis=1).drop('clouds', axis=1)

# Desmembrando a coluna 'weather' em colunas separadas
df_weather = pd.json_normalize(df['weather'].explode())
df_weather = df_weather.rename(columns=lambda x: f'weather_{x}')
df = pd.concat([df, df_weather], axis=1).drop('weather', axis=1)
df.head()

,dt,rain,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,gust,all,weather_id,weather_main,weather_description,weather_icon
0,1669334400,NaN,25.37,26.23,1013,87,25.37,25.37,0.93,18,1.11,93,804,Clouds,overcast clouds,04n
1,1669338000,NaN,24.25,25.03,1013,88,24.25,24.25,1.10,328,1.08,67,803,Clouds,broken clouds,04n
2,1669341600,NaN,20.39,20.81,1013,89,20.39,20.39,0.83,294,0.86,65,803,Clouds,broken clouds,04n
3,1669345200,NaN,24.25,25.08,1013,90,24.25,24.25,0.96,301,1.03,77,803,Clouds,broken clouds,04n
4,1669348800,NaN,24.25,25.11,1012,91,24.25,24.25,1.20,291,1.29,80,803,Clouds,broken clouds,04n


In [11]:
# Converter a coluna 'dt' para o formato de data/hora
df['dt'] = pd.to_datetime(df['dt'], unit='s')

# Converter 'dt' para o fuso horário de Brasília
df['dt'] = df['dt'].dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
df.head()

,dt,rain,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,gust,all,weather_id,weather_main,weather_description,weather_icon
0,2022-11-24 21:00:00-03:00,NaN,25.37,26.23,1013,87,25.37,25.37,0.93,18,1.11,93,804,Clouds,overcast clouds,04n
1,2022-11-24 22:00:00-03:00,NaN,24.25,25.03,1013,88,24.25,24.25,1.10,328,1.08,67,803,Clouds,broken clouds,04n
2,2022-11-24 23:00:00-03:00,NaN,20.39,20.81,1013,89,20.39,20.39,0.83,294,0.86,65,803,Clouds,broken clouds,04n
3,2022-11-25 00:00:00-03:00,NaN,24.25,25.08,1013,90,24.25,24.25,0.96,301,1.03,77,803,Clouds,broken clouds,04n
4,2022-11-25 01:00:00-03:00,NaN,24.25,25.11,1012,91,24.25,24.25,1.20,291,1.29,80,803,Clouds,broken clouds,04n


In [12]:
# Salvando o dataFrame
df.to_csv('dados_climaticos.csv', index=False)

In [13]:
# Filtrar os dados em um dia específico.
start_date = pd.Timestamp('2022-11-27 00:00:00').tz_localize('America/Sao_Paulo')
end_date = pd.Timestamp('2022-11-28 23:00:00').tz_localize('America/Sao_Paulo')

# Filtrar os dados dentro do intervalo de tempo especificado
filtered_data = df[df['dt'].between(start_date, end_date)]
filtered_data.head()

,dt,rain,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,gust,all,weather_id,weather_main,weather_description,weather_icon
51,2022-11-27 00:00:00-03:00,NaN,22.59,23.41,1013,96,22.59,22.59,0.94,281,1.07,76,803,Clouds,broken clouds,04n
52,2022-11-27 01:00:00-03:00,NaN,22.03,22.80,1013,96,22.03,22.03,1.18,292,1.61,76,803,Clouds,broken clouds,04n
53,2022-11-27 02:00:00-03:00,NaN,22.59,23.41,1012,96,22.59,22.59,1.32,282,2.03,78,803,Clouds,broken clouds,04n
54,2022-11-27 03:00:00-03:00,NaN,22.59,23.39,1012,95,22.59,22.59,1.01,286,1.35,77,803,Clouds,broken clouds,04n
55,2022-11-27 04:00:00-03:00,NaN,23.14,23.99,1012,95,23.14,23.14,1.01,289,1.27,83,803,Clouds,broken clouds,04n
